In [45]:
# CHECK MISSING
for p in list(Path("../outputs/multirun/ablations/").glob("*/*")):
    if not (p / "tb_logs.parquet").exists():
        print(p)

../outputs/multirun/ablations/amazon/anchoral-25-importance-random-True-50_2023-07-03T18-06-03_active_train_14
../outputs/multirun/ablations/amazon/anchoral-25-topk-kmeans-True-50_2023-07-03T16-01-00_active_train_18
../outputs/multirun/ablations/amazon/anchoral-25-topk-kmeans-True-50_2023-07-03T16-01-00_active_train_8
../outputs/multirun/ablations/amazon/anchoral-25-importance-kmeans-True-50_2023-07-03T18-06-03_active_train_52
../outputs/multirun/ablations/amazon/anchoral-25-topk-kmeans-True-50_2023-07-03T16-01-00_active_train_23
../outputs/multirun/ablations/amazon/anchoral-25-uniform-kmeans-True-50_2023-07-03T18-06-03_active_train_113
../outputs/multirun/ablations/amazon/anchoral-25-topk-kmeans-True-50_2023-07-03T16-01-00_active_train_3
../outputs/multirun/ablations/amazon/anchoral-25-topk-kmeans-True-50_2023-07-03T16-01-00_active_train_28
../outputs/multirun/ablations/amazon/anchoral-25-topk-kmeans-True-50_2023-07-03T16-01-00_active_train_33
../outputs/multirun/ablations/amazon/anch

In [39]:
list(
    Path(
        "../outputs/multirun/ablations/amazon/anchoral-25-importance-random-True-50_2023-07-03T18-06-03_active_train_14"
    ).iterdir()
)

[PosixPath('../outputs/multirun/ablations/amazon/anchoral-25-importance-random-True-50_2023-07-03T18-06-03_active_train_14/tb_logs'),
 PosixPath('../outputs/multirun/ablations/amazon/anchoral-25-importance-random-True-50_2023-07-03T18-06-03_active_train_14/.early_stopping.jsonl'),
 PosixPath('../outputs/multirun/ablations/amazon/anchoral-25-importance-random-True-50_2023-07-03T18-06-03_active_train_14/.hydra'),
 PosixPath('../outputs/multirun/ablations/amazon/anchoral-25-importance-random-True-50_2023-07-03T18-06-03_active_train_14/logs'),
 PosixPath('../outputs/multirun/ablations/amazon/anchoral-25-importance-random-True-50_2023-07-03T18-06-03_active_train_14/active_train.log'),
 PosixPath('../outputs/multirun/ablations/amazon/anchoral-25-importance-random-True-50_2023-07-03T18-06-03_active_train_14/hparams.yaml')]

In [ ]:
from itertools import combinations, permutations, product
from pathlib import Path

import duckdb
import matplotlib as mpl
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import polars as pl
import pygwalker as pyg
import seaborn as sns
import srsly
from matplotlib import ticker
from tqdm.auto import tqdm

In [ ]:
pl.Config.set_fmt_str_lengths(100)
con = duckdb.connect()

In [ ]:
mpl.rcParams["font.family"] = "monospace"  # "DejaVu Sans Mono"
plt.style.use("bmh")
sns.set_context("paper")
col_order = ["amazon", "pubmed", "eurlex", "agnews", "wiki_toxic"]

In [ ]:
hparams = []

for p in tqdm(list(Path("../outputs/multirun/").rglob("ablations/*/*/tb_logs.parquet"))):
    meta = srsly.read_yaml(p.parent / "hparams.yaml")
    hparams.append(
        dict(
            filename=str(p.parent),
            data_seed=meta["data"]["seed"],
            model_seed=meta["model"]["seed"],
            initial_seed=meta["active_data"]["seed"],
            global_seed=meta["seed"],
            retriever=meta["index_metric"],
            dataset=meta["dataset"]["short_name"],
            **meta["strategy"],
        )
    )

hparams_df = pd.DataFrame(hparams)

In [ ]:
unique_hparams = hparams_df.nunique()
changing_hparams = unique_hparams[unique_hparams > 1].index.tolist()
hparams_df = hparams_df.loc[:, changing_hparams]

In [ ]:
count_query = """
SELECT tag, count(tag)
FROM read_parquet('../outputs/multirun/ablations/*/*/tb_logs.parquet', filename=True)
GROUP BY tag
"""
metrics = con.execute(count_query).df()

In [ ]:
metrics.query("tag.str.contains('summary|search')")

In [ ]:
query = """
SELECT *
FROM read_parquet('../outputs/multirun/ablations/*/*/tb_logs.parquet', filename=True)
WHERE tag IN (
    'test/f1_class1',
    'test/recall_class1',
    'test/precision_class1',
    'test/loss',
    'summary/unique_ids_retrieved',
    'summary/cumulative_count_class_1',
    'summary/num_anchors',
    'search/unique_ids_retrieved',
    'summary/count_class_1',
)
"""

all_df = con.execute(query).df()
all_df = all_df.assign(
    filename=lambda _df: _df["filename"].map(lambda ex: str(Path(ex).parent)),
    auc=lambda _df: _df.groupby(["filename", "tag"])["value"].transform(np.trapz),
)

In [ ]:
df = pd.merge(all_df, hparams_df, on="filename", how="left")

In [ ]:
hparams = []

for p in tqdm(list(Path("../outputs/multirun/").rglob("test_new_kmeans/*/*/tb_logs.parquet"))):
    meta = srsly.read_yaml(p.parent / "hparams.yaml")
    hparams.append(
        dict(
            filename=str(p.parent),
            data_seed=meta["data"]["seed"],
            model_seed=meta["model"]["seed"],
            initial_seed=meta["active_data"]["seed"],
            global_seed=meta["seed"],
            retriever=meta["index_metric"],
            dataset=meta["dataset"]["short_name"],
            **meta["strategy"],
        )
    )

tmp_hparams_df = pd.DataFrame(hparams)


query = """
SELECT *
FROM read_parquet('../outputs/multirun/test_new_kmeans/*/*/tb_logs.parquet', filename=True)
WHERE tag IN (
    'test/f1_class1',
    'test/recall_class1',
    'test/precision_class1',
    'test/loss',
    'summary/unique_ids_retrieved',
    'summary/cumulative_count_class_1',
    'summary/num_anchors',
    'search/unique_ids_retrieved',
    'summary/count_class_1',
)
"""

tmp_all_df = con.execute(query).df()
tmp_all_df = tmp_all_df.assign(
    filename=lambda _df: _df["filename"].map(lambda ex: str(Path(ex).parent)),
    auc=lambda _df: _df.groupby(["filename", "tag"])["value"].transform(np.trapz),
)
tmp_df = pd.merge(tmp_all_df, tmp_hparams_df, on="filename", how="left")

In [ ]:
a = df.loc[
    (df["dataset"].isin(tmp_df["dataset"].unique()))
    & (df["subpool_size"].isin(tmp_df["subpool_size"].unique()))
    & (df["subpool_sampling_strategy"].isin(tmp_df["subpool_sampling_strategy"].unique()))
]

b = tmp_df.assign(anchor_strategy=lambda _df: _df["anchor_strategy"] + "_new")

c = pd.concat([a, b])
collate_col = sorted(
    [
        i
        for i in changing_hparams
        if "seed" not in i and i not in ("filename", "dataset", "anchor_strategy")
    ]
)
c["exp_id"] = c[collate_col].astype(str).T.agg("-".join)

In [ ]:
# CHECK THAT THE NUMBER OF MINORITY COINCIDES UP TO THE INITIAL BUDGET

# query = """
# SELECT filename, labelling_round as step, count(1) as num_minority
# FROM read_parquet('../outputs/multirun/ablations/*/*/*/labelled_dataset.parquet', filename=True)
# WHERE labels == 1
# GROUP BY filename, step
# """

# minority_count = con.execute(query).df()
# minority_count = minority_count.assign(
#     filename=lambda _df: _df["filename"].map(lambda ex: str(Path(ex).parents[1]))
# )

# (
#     pd.merge(
#         all_df.query("tag == 'summary/count_class_1'"),
#         minority_count.query("step >= 0").assign(step=lambda _df: _df["step"] - 1),
#         on=["filename", "step"],
#         how="inner",
#     )
#     .assign(diff=lambda _df: _df["value"] - _df["num_minority"])
#     ["diff"].sum()
# )

In [ ]:
collate_col = sorted(
    [i for i in changing_hparams if "seed" not in i and i not in ("filename", "dataset")]
)
df["exp_id"] = df[collate_col].astype(str).T.agg("-".join)

In [ ]:
check_df = df.groupby(["exp_id", "dataset"])["filename"].size().reset_index()
check_df.loc[check_df["filename"] < check_df["filename"].max()]

In [ ]:
loaded_metrics = df["tag"].unique().tolist()
loaded_metrics

In [ ]:
df.loc[df["anchor_strategy"] == "kmeans"][collate_col].drop_duplicates()

### Analaysis of AnchorAL

In [ ]:
metrics = [
    "summary/num_anchors",
    "search/unique_ids_retrieved",
    "summary/count_class_1",
    "test/f1_class1",
    # 'test/loss',
    "test/precision_class1",
    "test/recall_class1",
]

plot_data = df.loc[
    (df["tag"].isin(metrics))
    & (df["only_minority"] == True)
    & (df["subpool_sampling_strategy"] == "importance")
    & (df["subpool_size"] == 25)
    & (df["num_neighbours"] == 2000)
].assign(step=lambda _df: (_df["step"] * 25) + 100)

In [ ]:
hue_col = "exp_id"
legend_order = sorted(plot_data[hue_col].unique().tolist())
palette = dict(zip(legend_order, sns.color_palette("Set1", n_colors=len(legend_order))))

g = sns.FacetGrid(
    plot_data,
    row="tag",
    col="dataset",
    # col_wrap=3,
    sharex=True,
    legend_out=True,
    despine=True,
    # xlim=(0, xlim),
    # ylim=ylim,
    sharey=False,
    # col_order=col_order,
    margin_titles=True,
)
g.map_dataframe(
    sns.lineplot,
    data=plot_data,
    x="step",
    y="value",
    errorbar=("se", 1),
    hue=hue_col,
    palette=palette,
    style="tag",
)
g.set_axis_labels("", "")
g.set_titles(col_template="{col_name}", row_template="{row_name}")
g.add_legend(label_order=legend_order, bbox_to_anchor=(0.5, 1.02), title=hue_col)
g.tight_layout()
plt.show()

### Plot 2

In [ ]:
df.loc[
    (df["tag"].isin(metrics))
    & (df["subpool_size"].isin([25]))
    & (
        df["num_neighbours"].isin(
            [
                2000,
            ]
        )
    )
    & (df["only_minority"] == True)
].groupby(["dataset", "anchor_strategy"])["filename"].nunique()

In [ ]:
metrics = [
    "test/f1_class1",
    "summary/cumulative_count_class_1",
    "summary/num_anchors",
    "search/unique_ids_retrieved",
    # 'test/loss',
    # 'test/precision_class1',
    # 'test/recall_class1'
]

plot_data = df.loc[
    (df["tag"].isin(metrics))
    & (df["subpool_size"].isin([25]))
    & (
        df["num_neighbours"].isin(
            [
                2000,
            ]
        )
    )
    & (df["only_minority"] == True)
].assign(step=lambda _df: (_df["step"] * 25) + 100)

In [ ]:
plot_data = c.loc[
    (c["tag"].isin(metrics))
    & (c["subpool_size"].isin([25]))
    & (
        c["num_neighbours"].isin(
            [
                2000,
            ]
        )
    )
    & (c["only_minority"] == True)
    & (c["anchor_strategy"].isin(["kmeans", "kmeans_new"]))
].assign(step=lambda _df: (_df["step"] * 25) + 100)

In [ ]:
plot_data.groupby(["exp_id", "dataset"])["filename"].nunique()

In [ ]:
p = sns.relplot(
    data=plot_data,
    row="tag",
    col="dataset",
    x="step",
    y="value",
    hue="exp_id",
    style="anchor_strategy",
    kind="line",
    facet_kws=dict(sharex=True, sharey=False, margin_titles=True),
    errorbar=("se", 0),
)

In [ ]:
mpl.rc("figure", dpi=1000)

In [ ]:
df.loc[
    (df["dataset"] == "amazon")
    & (df["anchor_strategy"] == "kmeans")
    & (df["subpool_sampling_strategy"] == "topk")
    & (df["only_minority"] == True)
]

In [ ]:
hue_col = "exp_id"
style_col = "anchor_strategy"

# a = hue_col
# hue_col = style_col
# style_col = a

legend_order = sorted(plot_data[hue_col].unique().tolist())
palette = dict(zip(legend_order, sns.color_palette("bright", n_colors=len(legend_order))))

g = sns.FacetGrid(
    plot_data,
    row="tag",
    col="dataset",
    # col_wrap=3,
    sharex=True,
    legend_out=True,
    despine=True,
    # xlim=(0, xlim),
    # ylim=ylim,
    sharey=False,
    # col_order=col_order,
    margin_titles=True,
    row_order=metrics,
)
g.map_dataframe(
    sns.lineplot,
    data=plot_data,
    x="step",
    y="value",
    errorbar=("se", 0),
    hue=hue_col,
    palette=palette,
    style=style_col,
)
g.set_axis_labels("", "")
g.figure.subplots_adjust(hspace=0)
g.set_titles(col_template="{col_name}", row_template="{row_name}")
g.add_legend(label_order=legend_order, bbox_to_anchor=(0.5, 1.06), title=hue_col)
g.tight_layout()
plt.show()

In [ ]:
(
    plot_data.query("tag == 'test/f1_class1'")[["exp_id", "auc", "dataset"]]
    .drop_duplicates()
    .groupby(["exp_id", "dataset"])["auc"]
    .agg(["mean", "std"])
    .reset_index()
    .sort_values(["dataset", "mean"])
)

In [ ]:
(
    df.query("(tag == 'test/f1_class1') & (subpool_size == 25)")
    .groupby(["dataset", "exp_id", "filename"])["value"]
    .apply(np.trapz)
    .reset_index()
    .groupby(["dataset", "exp_id"])["value"]
    .agg(["mean", "std"])
    .sort_values(["dataset", "mean"])
)

In [ ]:
import hnswlib as hb
from datasets import load_from_disk
from hdbscan import HDBSCAN
from sklearn.preprocessing import StandardScaler
from umap import UMAP

In [ ]:
data = pd.read_parquet(
    "/home/pl487/anchoral/outputs/multirun/ablations/eurlex/anchoral-25-importance-kmeans-True-50_2023-07-03T18-06-03_active_train_140/logs/labelled_dataset.parquet"
)
ds = load_from_disk("/home/pl487/anchoral/data/prepared/eurlex-57k_bert-tiny/")["train"].to_pandas()

In [ ]:
uids = data.query("(labelling_round < 20) & (labels == 1)")["uid"].tolist()

In [ ]:
index = hb.Index(space="cosine", dim=364)
index.load_index("/home/pl487/anchoral/data/processed/eurlex-57k/all-MiniLM-L12-v2_cosine.bin")

In [ ]:
# uids = index.get_ids_list()
from sklearn.preprocessing import normalize

X = np.stack(index.get_items(uids))
X = normalize(X, axis=1)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

silhouette_avg_n_clusters = []
for n_clusters in range(2, len(uids)):

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=42, n_init="auto")
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    silhouette_avg_n_clusters.append(silhouette_avg)

np.argmax(silhouette_avg_n_clusters)

In [ ]:
plt.plot(silhouette_avg_n_clusters)

In [ ]:
plt.plot(silhouette_avg_n_clusters)

In [ ]:
from sklearn.preprocessing import Normalizer, StandardScaler

X = [[4, 1, 2, 2], [1, 3, 9, 3], [5, 7, 5, 1]]

Normalizer().fit_transform(X), StandardScaler().fit_transform(X)

In [ ]:
reducer = umap.UMAP(n_components=2)
scaled_emb = StandardScaler().fit_transform(emb)
scaled_emb = reducer.fit_transform(scaled_emb, ds["labels"])

In [ ]:
clusterer = HDBSCAN(min_cluster_size=500)
clusterer.fit(scaled_emb)

In [ ]:
len(set(clusterer.labels_))

In [ ]:
px.scatter(x=scaled_emb[:, 0], y=scaled_emb[:, 1], color=ds["labels"])

In [ ]:
px.scatter_3d(x=scaled_emb[:, 0], y=scaled_emb[:, 1], z=scaled_emb[:, 2], color=ds["labels"])